# Pipeline de Pré-processamento - Tic Tac Toe
**Author**: Gabriela Dellamora Paim, Bruno Duarte Carlan

**Version**: 12/04/2025

**Python Ver**: 3.12.9

In [ ]:
import pandas as pd

PATH_OLD = './data_old/tic-tac-toe.data'
PATH_NEW = './data_old/data_processed.csv'
PATH_ONGOING = './data_old/data_ongoing.csv'

X = 1
O = -1
BLANK = 0

O_WIN   = 0
DRAW    = 1
ONGOING = 2
X_WIN   = 3

In [ ]:
# Dataframe onde X sempre é quem começa
df_x_begins = pd.read_csv(PATH_OLD, index_col=False)
df_x_begins.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', 'category']
df_x_begins.describe()

# Tratar Dados Existentes

In [ ]:
# Para facilitar a diferenciacao entre X venceu e O venceu
df_x_begins.replace(to_replace={'x' : X, 'o' : O, 'b' : BLANK, 'positive' : X_WIN}, inplace=True)

In [ ]:
# Observações onde O começa
df_o_begins = df_x_begins.replace(to_replace={X:O, O:X, X_WIN:O_WIN})

df = pd.concat([df_x_begins, df_o_begins], ignore_index=True).copy()
print(f'BEFORE={df_x_begins.shape} : AFTER={df.shape}')

print(df.category.unique())
df.describe()

In [ ]:
def get_state(symbol):
    ''' Retorna simbolo utilizado pela pipeline.'''
    if symbol == O: return O_WIN
    if symbol == X: return X_WIN
    return DRAW

def check_wins(obs):
    '''Valida estado final do jogo. Pode ser O_WIN, X_WIN e DRAW. Trata casos que não precisam de alteração'''
    # lazy evaluation
    if obs.category != 'negative':
        return obs.category

    # Diagonais
    if (obs.iloc[0] == obs.iloc[4] == obs.iloc[8] != BLANK) or (obs.iloc[2] == obs.iloc[4] == obs.iloc[6] != BLANK):
        return get_state(obs.iloc[4])

    for i in range(3):
        # Linhas
        if obs.iloc[0 + i] == obs.iloc[3 + i] == obs.iloc[6 + i] != BLANK:
            return get_state(obs.iloc[0 + i])
        # Colunas
        if obs.iloc[0 + (i * 3)] == obs.iloc[1 + (i * 3)] == obs.iloc[2 + (i * 3)] != BLANK:
            return get_state(obs.iloc[0 + (i * 3)])
    return DRAW

In [ ]:
df['category'] = df.apply(check_wins, axis=1)

In [ ]:
print(df.category.unique())
print(f'Total={df.shape}')
print(f'Draw={df[df.category == DRAW].shape}')
df.describe()
df[df.category == DRAW].sample(10)

In [ ]:
# Exportar
df.to_csv(PATH_NEW, index=False)